In [1]:
#bibliotecas
import unidecode
from pyspark.sql.functions import udf,col,concat_ws, explode,length,abs, lower,lit
from pyspark.sql.types import StringType, IntegerType

#bibliotecas ML
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler,RegexTokenizer, StopWordsRemover, CountVectorizer, NGram
from pyspark.ml.classification import NaiveBayes,RandomForestClassifier,LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline,PipelineModel

#variables generales del notebook
stopWordsCustomizados = ["amlo","felipe calderon","EPN","calderon"] + StopWordsRemover.loadDefaultStopWords("spanish")
minTokenSize = 2
cantidadNGrams = 1

In [2]:
tweetData = spark.read.csv('dbfs:/mnt/jglake/tweetsSentiment.csv',sep=";",header="True")
tweetData.createOrReplaceTempView("tweetData")
display(tweetData)

usuario,texto,categoria
joselsanchez54,"AristeguiOnline @alvaro_delgado @paezvarela ¿Uds le darían alguna atención a un borrachín con delirium , apenas y puede pensar",negativo
benito06168625,No te preocupes mi felipon de todos modos la poli federal va para atr�s as� como el agrupamiento de granaderos de CDM as� lo dispone amlo y as� mismamente ser� el asunto es desaparecer lo que no ha funcionado,negativo
leticia15257127,"Eso si es apoyo a amlo, gracias",positivo
Fafhoo,Esta es la mano negra a la que se refería AMLO.La Policía Federal pide que Felipe Calderón sea su líder.Aho…,negativo
EarvDezeta,"AMLO es como el niño raro de la escuela que ahora está causando un tiroteo, los demás alumnos somos todos los mexicanos.",negativo
LorenzoMota64,"""Qué Pasa Aquí??! #AMLO #CUARTATRANSFORMACIÓN Cobran Boleto para Entrar a las Mañaneras??""""… """"""",informativo
azulMaruaLu,"¿Amlo sigue manejando que las protestas de la policia del gobierno neoliberal (PF), son un compló? 😬",informativo
consapm,"bueno que amlo no tiene ni la mas minima intencion de ir a algun foro, cumbre, lo que quieran ptm ya quiero ver cuando toque ir a la unga",negativo
feliztwit,"soy muy feliz, me gusta que amlo este cambiando al apis, gracias por todo , te amo cabezita de algod�n",positivo
LIBREDPENSAR,@luisghernan @lopezobrador_ @FelipeCalderon Y LOS MUERTOS DE AMLO ??????? YA SON MAS QUE LOS DE CALDERON EH. EN PRO…,negativo


In [3]:
%sql 
select categoria, count(*) as cantidad from tweetData group by categoria

categoria,cantidad
informativo,114
negativo,107
positivo,38


In [4]:
#Quitamos acentos y convertimos a minusculas
def remove_accents(input_str):
  return unidecode.unidecode(input_str).lower()

remove_accents_udf = udf(remove_accents, StringType())
tweetData = tweetData.withColumn('textoSinAcentos',remove_accents_udf(tweetData.texto))

#Tokenizamos
regexTokenizer = RegexTokenizer(inputCol="textoSinAcentos", outputCol="words", pattern="\\W")
#Establecemos token minimo
regexTokenizer.setMinTokenLength(minTokenSize)
#Quitamos stop words
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(stopWordsCustomizados)
#Convertimos a ngramas
ngram = NGram(n=cantidadNGrams, inputCol="filtered", outputCol="ngrams")
#Convertimos categoria de texto a numerica
label_stringIdx = StringIndexer(inputCol = "categoria", outputCol = "label")
#Vectorizamos
countVectors = CountVectorizer(inputCol="ngrams", outputCol="features", vocabSize=150, minDF=5)

#Ejecutamos el pipeline completo
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, ngram,countVectors,label_stringIdx])

pipelineFit = pipeline.fit(tweetData)
dfTweets = pipelineFit.transform(tweetData)
display(dfTweets)

usuario,texto,categoria,textoSinAcentos,words,filtered,ngrams,features,label
joselsanchez54,"AristeguiOnline @alvaro_delgado @paezvarela ¿Uds le darían alguna atención a un borrachín con delirium , apenas y puede pensar",negativo,"aristeguionline @alvaro_delgado @paezvarela ?uds le darian alguna atencion a un borrachin con delirium , apenas y puede pensar","List(aristeguionline, alvaro_delgado, paezvarela, uds, le, darian, alguna, atencion, un, borrachin, con, delirium, apenas, puede, pensar)","List(aristeguionline, alvaro_delgado, paezvarela, uds, darian, alguna, atencion, borrachin, delirium, apenas, puede, pensar)","List(aristeguionline, alvaro_delgado, paezvarela, uds, darian, alguna, atencion, borrachin, delirium, apenas, puede, pensar)","List(0, 50, List(32), List(1.0))",1.0
benito06168625,No te preocupes mi felipon de todos modos la poli federal va para atr�s as� como el agrupamiento de granaderos de CDM as� lo dispone amlo y as� mismamente ser� el asunto es desaparecer lo que no ha funcionado,negativo,no te preocupes mi felipon de todos modos la poli federal va para atrs as como el agrupamiento de granaderos de cdm as lo dispone amlo y as mismamente ser el asunto es desaparecer lo que no ha funcionado,"List(no, te, preocupes, mi, felipon, de, todos, modos, la, poli, federal, va, para, atrs, as, como, el, agrupamiento, de, granaderos, de, cdm, as, lo, dispone, amlo, as, mismamente, ser, el, asunto, es, desaparecer, lo, que, no, ha, funcionado)","List(preocupes, felipon, modos, poli, federal, va, atrs, as, agrupamiento, granaderos, cdm, as, dispone, as, mismamente, ser, asunto, desaparecer, funcionado)","List(preocupes, felipon, modos, poli, federal, va, atrs, as, agrupamiento, granaderos, cdm, as, dispone, as, mismamente, ser, asunto, desaparecer, funcionado)","List(0, 50, List(4, 25, 28), List(1.0, 1.0, 1.0))",1.0
leticia15257127,"Eso si es apoyo a amlo, gracias",positivo,"eso si es apoyo a amlo, gracias","List(eso, si, es, apoyo, amlo, gracias)","List(si, apoyo, gracias)","List(si, apoyo, gracias)","List(0, 50, List(1, 10), List(1.0, 1.0))",2.0
Fafhoo,Esta es la mano negra a la que se refería AMLO.La Policía Federal pide que Felipe Calderón sea su líder.Aho…,negativo,esta es la mano negra a la que se referia amlo.la policia federal pide que felipe calderon sea su lider.aho...,"List(esta, es, la, mano, negra, la, que, se, referia, amlo, la, policia, federal, pide, que, felipe, calderon, sea, su, lider, aho)","List(mano, negra, referia, policia, federal, pide, felipe, lider, aho)","List(mano, negra, referia, policia, federal, pide, felipe, lider, aho)","List(0, 50, List(4, 5, 22), List(1.0, 1.0, 1.0))",1.0
EarvDezeta,"AMLO es como el niño raro de la escuela que ahora está causando un tiroteo, los demás alumnos somos todos los mexicanos.",negativo,"amlo es como el nino raro de la escuela que ahora esta causando un tiroteo, los demas alumnos somos todos los mexicanos.","List(amlo, es, como, el, nino, raro, de, la, escuela, que, ahora, esta, causando, un, tiroteo, los, demas, alumnos, somos, todos, los, mexicanos)","List(nino, raro, escuela, ahora, causando, tiroteo, demas, alumnos, mexicanos)","List(nino, raro, escuela, ahora, causando, tiroteo, demas, alumnos, mexicanos)","List(0, 50, List(17), List(1.0))",1.0
LorenzoMota64,"""Qué Pasa Aquí??! #AMLO #CUARTATRANSFORMACIÓN Cobran Boleto para Entrar a las Mañaneras??""""… """"""",informativo,"""que pasa aqui??! #amlo #cuartatransformacion cobran boleto para entrar a las mananeras??""""... """"""","List(que, pasa, aqui, amlo, cuartatransformacion, cobran, boleto, para, entrar, las, mananeras)","List(pasa, aqui, cuartatransformacion, cobran, boleto, entrar, mananeras)","List(pasa, aqui, cuartatransformacion, cobran, boleto, entrar, mananeras)","List(0, 50, List(), List())",0.0
azulMaruaLu,"¿Amlo sigue manejando que las protestas de la policia del gobierno neoliberal (PF), son un compló? 😬",informativo,"?amlo sigue manejando que las protestas de la policia del gobierno neolibe

In [5]:
# Dividiendo datos de entrenamiento y de prueba
(trainingData, testData) = dfTweets.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

#algoritmo naive bayes
nb = NaiveBayes(smoothing=1)
#entrenamiento
modelNB = nb.fit(trainingData)
#generar predicciones
predictions = modelNB.transform(testData)
predictions.select("texto","categoria","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 20, truncate = 20)

#evaluando el modelo
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
precision = evaluator.evaluate(predictions)
print("Precisión del modelo: "+ str(precision))

Training Dataset Count: 177
Test Dataset Count: 82
+--------------------+-----------+--------------------+-----+----------+
 texto| categoria| probability|label|prediction|
+--------------------+-----------+--------------------+-----+----------+
Niega Calderón es...|informativo|[0.98419175484670...| 0.0| 0.0|
Por ley desaparec...|informativo|[0.97870169717071...| 0.0| 0.0|
Ver ¡DE ÚLTIMA HO...|informativo|[0.97870169717071...| 0.0| 0.0|
....Ver ¡DE ÚLTIM...|informativo|[0.97265972234058...| 0.0| 0.0|
En 18 meses desap...|informativo|[0.95198137209931...| 0.0| 0.0|
PROTESTAS EN LA P...|informativo|[0.89957298145949...| 0.0| 0.0|
Esta es la mano n...| negativo|[0.76092034452344...| 1.0| 0.0|
bueno que amlo no...| negativo|[0.75760015869431...| 1.0| 0.0|
Arremete AMLO con...| negativo|[0.73773062682014...| 1.0| 0.0|
@susyfiona1 @Roan...| negativo|[0.70051264001882...| 1.0| 0.0|
@AdrianGLara @my_...| negativo|[0.63692861671349...| 1.0| 0.0|
Busca AMLO cerrar...|informativo|[0.63210799109535...| 0.0| 0.0|
HACEN EL RIDÍCULO...| negativo|[0.62072299082336...| 1.0| 0.0|
@lejournallatino ...|informativo|[0.60709612683278...| 0.0| 0.0|
#Mañanera #AMLO 4...|informativo|[0.60213518862724...| 0.0| 0.0|
📹 Noroña se destapa|informativo|[0.59143003255670...| 0.0| 0.0|
Con Andrés Manuel...| negativo|[0.58768503510891...| 1.0| 0.0|
"@MalthusGamba @l...|informativo|[0.54338011979097...| 0.0| 0.0|
AMLO, “deje de di...| negativo|[0.53906845236953...| 1.0| 0.0|
@leozuckermann Cu...|informativo|[0.52053797182311...| 0.0| 0.0|
+--------------------+-----------+--------------------+-----+----------+
only showing top 20 rows

Precisión del modelo: 0.6038233355306526

In [6]:
#usando random forest
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
# Train model with Training Data
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.select("texto","categoria","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 20, truncate = 30)

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
precision = evaluator.evaluate(predictions)
print("Precisión del modelo: "+ str(precision))


+------------------------------+-----------+------------------------------+-----+----------+
 texto| categoria| probability|label|prediction|
+------------------------------+-----------+------------------------------+-----+----------+
Niega Calderón estar detrás...|informativo|[0.7341302050198677,0.20723...| 0.0| 0.0|
Por ley desaparecerá Policí...|informativo|[0.6963802283282924,0.23598...| 0.0| 0.0|
....Ver ¡DE ÚLTIMA HORA! CA...|informativo|[0.6815727608738016,0.24870...| 0.0| 0.0|
Ver ¡DE ÚLTIMA HORA! CALDER...|informativo|[0.6766947120933138,0.25510...| 0.0| 0.0|
En 18 meses desaparece la p...|informativo|[0.674701097064387,0.253757...| 0.0| 0.0|
Esta es la mano negra a la ...| negativo|[0.6174796529232398,0.27708...| 1.0| 0.0|
PROTESTAS EN LA PF CONTRA A...|informativo|[0.6019106323126222,0.29862...| 0.0| 0.0|
Con Andrés Manuel López Obr...| negativo|[0.5198250397427157,0.36677...| 1.0| 0.0|
Arremete AMLO contra Chong ...| negativo|[0.5196047105299645,0.36590...| 1.0| 0.0|
#Mañanera #AMLO 4-Jul-19. L...|informativo|[0.5194033682480853,0.36832...| 0.0| 0.0|
"Y la última pero no menos ...| positivo|[0.5076461619305953,0.37358...| 2.0| 0.0|
HACEN EL RIDÍCULO ! PF es m...| negativo|[0.5022804269085094,0.36661...| 1.0| 0.0|
@susyfiona1 @RoanaBuenavent...| negativo|[0.5013418867944953,0.37546...| 1.0| 0.0|
@AdrianGLara @my_Mitchel @P...| negativo|[0.49933331471830555,0.3754...| 1.0| 0.0|
bueno que amlo no tiene ni ...| negativo|[0.4966968597720923,0.38619...| 1.0| 0.0|
 📹 Noroña se destapa|informativo|[0.49233937834780844,0.3822...| 0.0| 0.0|
Busca AMLO cerrar oficinas ...|informativo|[0.4890583113044532,0.38102...| 0.0| 0.0|
@lejournallatino @NoticiasM...|informativo|[0.48455057387072953,0.3738...| 0.0| 0.0|
¿Quién está metiendo la man...|informativo|[0.4824050168228413,0.40925...| 0.0| 0.0|
"@MalthusGamba @leozuckerma...|informativo|[0.48219475580122406,0.3876...| 0.0| 0.0|
+------------------------------+-----------+------------------------------+-----+----------+
only showing top 20 rows

Precisión del modelo: 0.5814867952601095

In [7]:
#usando Logistic Regression using Count Vector Features
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.select("texto","categoria","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 20, truncate = 20)

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
precision = evaluator.evaluate(predictions)
print("Precisión del modelo: "+ str(precision))

+--------------------+-----------+--------------------+-----+----------+
 texto| categoria| probability|label|prediction|
+--------------------+-----------+--------------------+-----+----------+
....Ver ¡DE ÚLTIM...|informativo|[0.89043064566261...| 0.0| 0.0|
Ver ¡DE ÚLTIMA HO...|informativo|[0.87879906899615...| 0.0| 0.0|
Por ley desaparec...|informativo|[0.86276675423305...| 0.0| 0.0|
Niega Calderón es...|informativo|[0.85154307970261...| 0.0| 0.0|
PROTESTAS EN LA P...|informativo|[0.80700247284348...| 0.0| 0.0|
bueno que amlo no...| negativo|[0.80526539054340...| 1.0| 0.0|
En 18 meses desap...|informativo|[0.77176748284392...| 0.0| 0.0|
@susyfiona1 @Roan...| negativo|[0.75734433268289...| 1.0| 0.0|
📹 Noroña se destapa|informativo|[0.68325311776513...| 0.0| 0.0|
@abrahamendieta Y...|informativo|[0.67549385370739...| 0.0| 0.0|
"@MalthusGamba @l...|informativo|[0.63614084490550...| 0.0| 0.0|
@abrahamendieta Y...|informativo|[0.63082240771434...| 0.0| 0.0|
@AdrianGLara @my_...| negativo|[0.61408491306064...| 1.0| 0.0|
Busca AMLO cerrar...|informativo|[0.59390299653932...| 0.0| 0.0|
@leozuckermann Cu...|informativo|[0.56862785136346...| 0.0| 0.0|
Arremete AMLO con...| negativo|[0.55125135655593...| 1.0| 0.0|
Responde el exPre...|informativo|[0.54671263796186...| 0.0| 0.0|
AMLO, “deje de di...| negativo|[0.53227560512712...| 1.0| 0.0|
"Y la última pero...| positivo|[0.52054149250696...| 2.0| 0.0|
@SergioSarmiento ...|informativo|[0.51874193484148...| 0.0| 0.0|
+--------------------+-----------+--------------------+-----+----------+
only showing top 20 rows

Precisión del modelo: 0.6131037780966044